# Try MWAS with covariates prepared in various ways

# Pick the regions we will test

In [1]:
library(data.table)

prev_hits <- fread("
Chr     pos        old_z       old_p
11   38247902        -27.1535308285104       2.30024742330298e-162
2    47933357        18.3327793004811        4.53147699327216e-75
7    1987910         10.1255085321387        4.25759742738181e-24
7    1987896         10.1255085321387        4.25759742738189e-24
7    1987797         10.0241523897721        1.19379483108027e-23
7    1987778         10.0105851568856        1.36940148731912e-23
12   2194742         -10.0072634920486       1.41615523554955e-23
")


In [2]:
#df <- fread("09.5-OUT_matched_SNP_meth_cov_chunked_EXPANSE_a2.csv")

In [3]:
df <- fread("09-OUT_matched_SNP_meth_cov_a2.csv")

## Try original code

In [4]:
###### model: learn elastic net model on training data 
######---------Input: trainX, trainY
######---------Return: selected features and coefficents

elastic.net <- function(trainX,trainY){
    if(nrow(trainX)!=length(trainY)){
            stop("Number of observations is differerent")
    } 

    # optimize alpha---mixing parameter  
    a <- 0.5
    search <- foreach(ai = a, .combine = rbind) %dopar% {
        cv.fit <- cv.glmnet(
                        trainX,
                        trainY,
                        nfold = 5,
                        type.measure = "mse",
                        paralle = TRUE,
                        alpha = ai
                        )
        data.frame(
                        cvm = min(cv.fit$cvm),
                        lambda = cv.fit$lambda.min,
                        alpha = ai
                        )
        } 
    cv.opt <- search[search$cvm == min(search$cvm),] 

        # fit model by optimized alpha and lambda
        yfit = glmnet(
        trainX,
        trainY,
        lambda = cv.opt$lambda,
        alpha = cv.opt$alpha
                )       
        idf <- coef(yfit)
        idx <- which(idf != 0)
        selectf <- data.frame(
                features = idf@Dimnames[[1]][idx], 
                coefs = idf [idx]
        )
}

MWAS <- function(gwas, weight, geno){
        z <- gwas %*% weight
        z.cor <- cor(geno)
        se <- sqrt(weight %*%  z.cor %*%  weight)
        z <- z/se
        p=pnorm(abs(z),lower.tail=F)*2
        return(c(z, p))
}

In [5]:
df <- df[which(df$Chr == 7), ]

In [6]:
i <- 2

In [7]:
library("glmnet")
library("e1071")
library("doParallel")

set.seed(2018)
wind <- c(5000,10000)
# output directory
#outd <- "/dcl02/lieber/shan/shizhong/finemapping/GWAS/tags/scz3/mwas/chr22/1/"
outd <- "20-OUT_original_mwas_sanity_test/"

Loading required package: Matrix

Loaded glmnet 4.1-8

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



In [8]:
suppressWarnings(library(bsseq))

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:Matrix’:

    expand, unname


The following objects are masked from ‘package:data.table’:

    first, second


The following object is masked from ‘package:utils’:

    findMatches


The

In [9]:
# load data for mwas
# load("./rda/caudate_mwas_data_chr22.rda")
load(df$methylation_data[i])

p <- getMeth(BSobj2)


rownames(p) <- start(BSobj2)

sites_to_test <- which(start(BSobj2) >= (1987910 - 500) & start(BSobj2) <= (1987910 + 500))
p <- p[sites_to_test, ]

# candidate cg
cg <- as.numeric(rownames(p))

# regress out covariates
#load("covs_for_meqtl.rda")

In [24]:
load("BSsample.rda")

In [27]:
head(BSsample)

,ID,brnum,has_genotype,source,agedeath,sex,race,primarydx,pmi,ph
,<chr>,<chr>,<lgl>,<fct>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<dbl>
1,LIBD1633_190509,Br0836,TRUE,DC,20.77,M,AA,Control,25.5,6.46
2,LIBD1654_190509,Br0845,TRUE,DC,24.79,M,AA,Control,43.5,6.94
3,LIBD1508_190509,Br0848,TRUE,DC,22.71,M,AA,Control,38.5,6.57
4,LIBD1655_190509,Br0863,TRUE,DC,43.85,M,AA,Control,24.5,6.57
5,LIBD1604_190509,Br0914,TRUE,DC,19.69,F,AA,Control,19.0,6.98
6,LIBD1389_190509,Br0948,TRUE,DC,45.91,M,AA,Control,38.5,6.66


In [26]:
BSsample[1:10, 1:10]

,ID,brnum,has_genotype,source,agedeath,sex,race,primarydx,pmi,ph
,<chr>,<chr>,<lgl>,<fct>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<dbl>
1,LIBD1633_190509,Br0836,TRUE,DC,20.77,M,AA,Control,25.5,6.46
2,LIBD1654_190509,Br0845,TRUE,DC,24.79,M,AA,Control,43.5,6.94
3,LIBD1508_190509,Br0848,TRUE,DC,22.71,M,AA,Control,38.5,6.57
4,LIBD1655_190509,Br0863,TRUE,DC,43.85,M,AA,Control,24.5,6.57
5,LIBD1604_190509,Br0914,TRUE,DC,19.69,F,AA,Control,19.0,6.98
6,LIBD1389_190509,Br0948,TRUE,DC,45.91,M,AA,Control,38.5,6.66
7,LIBD1675_190509,Br0949,TRUE,DC,18.05,M,AA,Control,14.5,6.97
8,LIBD1470_190509,Br0963,TRUE,DC,62.63,F,AA,Schizo,76.0,6.38
9,LIBD1434_190509,Br0983,TRUE,DC,43.38,M,CAUC,Schizo,25.0,6.37


In [25]:
names(attributes(BSsample))

[1] "names"     "class"     "row.names"

In [10]:
covs <- fread(df$cov_file[i])
covs <- t(covs)
colnames(covs) <- covs[1, ]
covs <- covs[2:nrow(covs), ]
# transpose so we have same orientation as original code

In [11]:
mat <- match(BSobj2$brnum,colnames(covs)) 
covs <- t(covs[,mat])
p.residual=matrix(NA,dim(p)[1],dim(p)[2])

In [12]:
rownames(covs)[is.na(covs[, 'genoPC1'])] <- BSobj2$brnum[is.na(covs[, 'genoPC1'])]

In [13]:
colnames(p.residual) <- BSobj2$brnum

In [14]:
covs <- as.data.frame(covs)
# Convert all columns except Dx and Sex from character to numeric
cols_to_convert <- setdiff(names(covs), c("Dx", "Sex"))

for (col in cols_to_convert) {
  covs[[col]] <- as.numeric(covs[[col]])
}

# Print the modified data frame to check the conversion
#print(dat)


In [15]:
for (i in 1:dim(p)[1]) { # For each methylation site
    dat <- as.data.frame(cbind(y = p[i,], covs))
    
    # Check for rows with NAs (the ones for which we don't have covariate data)
    valid_rows <- complete.cases(dat)
    
    if (sum(valid_rows) > 0) {
        dat_valid <- dat[valid_rows,]
        model.res <- lm(y ~ ., data = dat_valid)
        
        # Store residuals in the corresponding positions
        p.residual[i, valid_rows] <- resid(model.res)
    }
}


# for(i in 1:dim(p)[1]){ # foro each methylation site
#         dat <- as.data.frame(cbind(p[i,],covs))
#         colnames(dat) <- c("y",paste0("x",1:ncol(covs)))
#         model.res <- lm(reformulate(paste0("x",1:ncol(covs)), "y"),dat)
#         p.residual[i,] = resid(model.res) 
# }

In [16]:
inspect <- p.residual
colnames(inspect) <- gsub("Br0", "Br", colnames(inspect))
inspect <- inspect[, order(colnames(inspect))]

In [17]:
inspect[1:10, 1:10]

Br1003,Br1004,Br1007,Br1016,Br1017,Br1021,Br1023,Br1030,Br1033,Br1034
0.03257891,0.01985847,0.01691957,0.0060012777,-0.007263415,-0.023456398,-0.02982697,0.0228373821,0.03297282,0.02401404
0.03215905,0.01967398,0.01716213,0.0057195011,-0.007217071,-0.023042783,-0.02957225,0.0220174331,0.03277163,0.02373302
0.03077852,0.01906655,0.01791941,0.0048094823,-0.007028945,-0.021680599,-0.02866728,0.0193969365,0.03208693,0.02281625
0.02943789,0.01847816,0.01859645,0.0039506396,-0.006802293,-0.020357128,-0.02770284,0.0169500697,0.03139325,0.02193598
0.02861438,0.01811793,0.01898610,0.0034351395,-0.006645031,-0.019545627,-0.02707460,0.0154904553,0.03095493,0.02139836
0.02768335,0.01771243,0.01940548,0.0028635353,-0.006452723,-0.018631372,-0.02633618,0.0138785303,0.03044998,0.02079229
0.02539013,0.01672488,0.02035221,0.0015062943,-0.005922150,-0.016399324,-0.02440937,0.0100654824,0.02916825,0.01930539
0.02352718,0.01593283,0.02104292,0.0004543871,-0.005443023,-0.014612172,-0.02274988,0.0071180509,0.02808936,0.01810355
0.02193501,0.01526376,0.02158426,-0.0004087996,-0.005005569,-0.013106181,-0.02127722,0.0046961344,0.02714190,0.01708005
0.01811024,0.01368837,0.02271951,-0.0023449314,-0.003865747,-0.009569197,-0.01757639,-0.0007923371,0.02477187,0.01462124


In [22]:
dim(BSsample)

[1] 308  10

In [23]:
BSsample[1:10, 1:10]

DataFrame with 10 rows and 10 columns
                ID       brnum has_genotype   source  agedeath      sex
       <character> <character>    <logical> <factor> <numeric> <factor>
1  LIBD1633_190509      Br0836         TRUE       DC     20.77        M
2  LIBD1654_190509      Br0845         TRUE       DC     24.79        M
3  LIBD1508_190509      Br0848         TRUE       DC     22.71        M
4  LIBD1655_190509      Br0863         TRUE       DC     43.85        M
5  LIBD1604_190509      Br0914         TRUE       DC     19.69        F
6  LIBD1389_190509      Br0948         TRUE       DC     45.91        M
7  LIBD1675_190509      Br0949         TRUE       DC     18.05        M
8  LIBD1470_190509      Br0963         TRUE       DC     62.63        F
9  LIBD1434_190509      Br0983         TRUE       DC     43.38        M
10 LIBD1597_190509      Br0991         TRUE       DC     35.24        M
       race primarydx       pmi        ph
   <factor>  <factor> <numeric> <numeric>
1      AA     

In [21]:
getMeth(BSsample)

ERROR: Error in getMeth(BSsample): is(BSseq, "BSseq") is not TRUE


In [19]:
names(attributes(BSsample))

[1] "rownames"        "nrows"           "listData"        "elementType"    
[5] "elementMetadata" "metadata"        "class"

In [18]:
load("p1.rda", verbose = TRUE)

Loading objects:
  snp.gwas2
  snp.1kg.eur2
  map.1kg.eur2
  snp2
  map2
  p
  BSsample


In [19]:
wind <- 10000

In [20]:
# built predition models
idx.ea <- BSobj2$race == "CAUC"

In [21]:
snp2 <- snp2[, colnames(snp2) %in% colnames(p.residual)]

In [22]:
library(CpGWAS)

In [23]:
df$SNP_data

[1] "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas//libd_chr7.pgen"
[2] "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas//libd_chr7.pgen"
[3] "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas//libd_chr7.pgen"
[4] "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas//libd_chr7.pgen"
[5] "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas//libd_chr7.pgen"
[6] "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas//libd_chr7.pgen"
[7] "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas//libd_chr7.pgen"
[8] "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas//libd_chr7.pgen"
[9] "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas//libd_chr7.pgen"

In [24]:
i <- 2

In [25]:
snp_data <- loadSNPData(gsub("pgen", "pvar", df$SNP_data[i]),
                        df$SNP_data[i],
                        gsub("pgen", "psam", df$SNP_data[i]))

In [26]:
snp2_backup <- snp2

In [27]:
colnames(snp2) <- gsub("Br0", "Br", colnames(snp2))

In [28]:
snp2[1:10, 1:10]

,Br836,Br845,Br848,Br863,Br914,Br948,Br949,Br963,Br983,Br991
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr7:1963098:T:C,2.000,2,2.000,2,2.000,1.999,2.000,1.974,1.000,2
chr7:1963408:C:T,0.001,2,1.991,0,1.969,0.006,0.004,0.140,2.000,0
chr7:1964786:C:T,2.000,2,1.000,2,2.000,2.000,2.000,2.000,1.000,2
chr7:1966112:T:C,2.000,2,1.001,2,2.000,2.000,2.000,2.000,1.002,2
chr7:1973362:G:A,2.000,2,1.095,2,2.000,2.000,0.004,2.000,1.002,2
chr7:1975412:T:G,2.000,2,2.000,2,2.000,2.000,2.000,2.000,1.000,2
chr7:1976335:C:T,0.000,2,1.991,0,1.980,0.024,2.000,0.015,2.000,0
chr7:1977810:C:T,0.000,1,1.000,0,0.000,0.000,2.000,0.000,1.000,0
chr7:1978402:C:T,0.008,2,1.991,0,1.981,0.024,2.000,0.015,2.000,0


In [29]:
dim(snp2)

[1]  56 297

Why do we have 56 here but less in RStudio environment?

In [30]:
length(which(snp_data$pvar_dt$ID %in% rownames(snp2)))

[1] 56

In [31]:
rownames(snp2)

[1] "chr7:1963098:T:C" "chr7:1963408:C:T" "chr7:1964786:C:T" "chr7:1966112:T:C"
 [5] "chr7:1973362:G:A" "chr7:1975412:T:G" "chr7:1976335:C:T" "chr7:1977810:C:T"
 [9] "chr7:1978402:C:T" "chr7:1978783:G:T" "chr7:1979188:C:A" "chr7:1979235:G:A"
[13] "chr7:1980240:G:A" "chr7:1981269:G:C" "chr7:1981360:G:A" "chr7:1982428:C:G"
[17] "chr7:1985366:G:C" "chr7:1985405:A:G" "chr7:1987676:G:T" "chr7:1987689:G:A"
[21] "chr7:1987719:C:T" "chr7:1987813:T:C" "chr7:1989028:A:C" "chr7:1989333:G:T"
[25] "chr7:1989994:G:C" "chr7:1990232:T:C" "chr7:1990305:T:C" "chr7:1990372:G:C"
[29] "chr7:1990469:G:A" "chr7:1990652:G:T" "chr7:1993168:C:T" "chr7:1993230:A:T"
[33] "chr7:1994145:G:C" "chr7:1994382:C:T" "chr7:1994558:A:G" "chr7:1994714:G:A"
[37] "chr7:1994927:A:G" "chr7:1997034:T:G" "chr7:1997580:G:A" "chr7:1998126:G:A"
[41] "chr7:1999959:A:C" "chr7:2001797:A:G" "chr7:2002003:G:A" "chr7:2002813:C:T"
[45] "chr7:2002912:C:G" "chr7:2004461:G:C" "chr7:2005716:T:C" "chr7:2006553:G:C"
[49] "chr7:2006693:A:G" "chr7:2007195:C:T" "chr7:2008210:A:G" "chr7:2008240:G:A"
[53] "chr7:2008501:T:C" "chr7:2008585:G:C" "chr7:2008700:A:G" "chr7:2009071:A:G"

In [32]:
min(rownames(snp2))

[1] "chr7:1963098:T:C"

In [33]:
SNPs <- pgenlibr::ReadList(snp_data$pgen,
                           which(snp_data$pvar_dt$ID %in% rownames(snp2)))

In [34]:
genotype_IDs <- colnames(snp2)

In [35]:
#genotype_IDs <- gsub("Br0", "", genotype_IDs)

In [36]:
colnames(SNPs) <- rownames(snp2)

In [37]:
rownames(SNPs) <- snp_data$psam$`#IID`

In [38]:
# Filter out rows from geno that are not in genotype_IDs
idx_geno <- which(row.names(SNPs) %in% genotype_IDs)
geno_filtered <- SNPs[idx_geno, ]

In [39]:
dim(geno_filtered)

[1] 297  56

In [40]:
# Filter out rows from geno that are not in genotype_IDs
idx_geno <- which(row.names(SNPs) %in% genotype_IDs)
geno_filtered <- SNPs[idx_geno, ]

# Check if both geno and genotype_IDs have the same row names
if (length(genotype_IDs) != nrow(geno_filtered) || !all(genotype_IDs %in% row.names(geno_filtered))) {
  missing_genotype_IDs <- setdiff(genotype_IDs, row.names(geno_filtered))
  stop(paste("Missing genotypes:", paste(missing_genotype_IDs, collapse = ", ")))
}

# Find the matching indices for reordering
match_indices <- match(genotype_IDs, row.names(geno_filtered))

# Reorder geno matrix to match the order of genotype_IDs
reordered_geno <- geno_filtered[match_indices, ]

# Edge case where we only have one SNP in window
if (!is.matrix(geno_filtered)) {
  geno_filtered <- as.matrix(geno_filtered)
}
SNPs <- geno_filtered

In [41]:
SNPs <- t(SNPs)

In [42]:
SNPs <- SNPs[, colnames(snp2)]

In [43]:
SNPs[1:10, 1:10]

,Br836,Br845,Br848,Br863,Br914,Br948,Br949,Br963,Br983,Br991
chr7:1963098:T:C,0.000000,0,0.000000000,0,0.00000000,0.0009765625,0.000000,0.02600098,1.0000000,0
chr7:1963408:C:T,1.999023,0,0.008972168,2,0.03100586,1.9940185547,1.995972,1.85998535,0.0000000,2
chr7:1964786:C:T,0.000000,0,1.000000000,0,0.00000000,0.0000000000,0.000000,0.00000000,1.0000000,0
chr7:1966112:T:C,0.000000,0,0.999023438,0,0.00000000,0.0000000000,0.000000,0.00000000,0.9979858,0
chr7:1973362:G:A,0.000000,0,0.905029297,0,0.00000000,0.0000000000,1.995972,0.00000000,0.9979858,0
chr7:1975412:T:G,0.000000,0,0.000000000,0,0.00000000,0.0000000000,0.000000,0.00000000,1.0000000,0
chr7:1976335:C:T,2.000000,0,0.008972168,2,0.02001953,1.9760131836,0.000000,1.98498535,0.0000000,2
chr7:1977810:C:T,2.000000,1,1.000000000,2,2.00000000,2.0000000000,0.000000,2.00000000,1.0000000,2
chr7:1978402:C:T,1.992004,0,0.008972168,2,0.01898193,1.9760131836,0.000000,1.98498535,0.0000000,2
chr7:1978783:G:T,2.000000,0,0.008972168,2,0.02099609,1.9760131836,0.000000,1.98101807,0.0000000,2


In [44]:
snp2[1:10, 1:10]

,Br836,Br845,Br848,Br863,Br914,Br948,Br949,Br963,Br983,Br991
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr7:1963098:T:C,2.000,2,2.000,2,2.000,1.999,2.000,1.974,1.000,2
chr7:1963408:C:T,0.001,2,1.991,0,1.969,0.006,0.004,0.140,2.000,0
chr7:1964786:C:T,2.000,2,1.000,2,2.000,2.000,2.000,2.000,1.000,2
chr7:1966112:T:C,2.000,2,1.001,2,2.000,2.000,2.000,2.000,1.002,2
chr7:1973362:G:A,2.000,2,1.095,2,2.000,2.000,0.004,2.000,1.002,2
chr7:1975412:T:G,2.000,2,2.000,2,2.000,2.000,2.000,2.000,1.000,2
chr7:1976335:C:T,0.000,2,1.991,0,1.980,0.024,2.000,0.015,2.000,0
chr7:1977810:C:T,0.000,1,1.000,0,0.000,0.000,2.000,0.000,1.000,0
chr7:1978402:C:T,0.008,2,1.991,0,1.981,0.024,2.000,0.015,2.000,0


In [45]:
library(stringr)

In [46]:
for(k in 1:length(wind)){
    models.ea <- c()
    models.all <- c()
    for(i in 1:length(cg)){
            cat(i,"\n")
            range1 <- ifelse(cg[i] - wind[k] > 0,cg[i] - wind[k],0)
            range2 <- cg[i] + wind[k]
            idx <- map2$POS > range1 & map2$POS < range2
            # go to next cg if no snps within window
            if(sum(idx) <= 1){
                    next
            }
            #geno <- snp2[idx,]
            geno <- snp2[idx,]
            rownames(geno) <- map2$POS[idx]
            trainX <- t(geno)
            trainY <- p.residual[i,]
            fit <- tryCatch(
                    elastic.net(trainX,trainY),
                    error = function(e) {return ("err")})
            if(!is.data.frame(fit)){
                if(fit == "err"){
                    next
                }
            }
            if(nrow(fit) == 0) next

            fit$cg <- cg[i]
            models.all <- rbind(models.all,fit)
            # EA only
            trainX <- trainX[idx.ea,]
            if(sum(apply(trainX,2,var)!=0) <= 1){
                    next
            }
            trainY <- trainY[idx.ea]
            fit <- tryCatch(
                    elastic.net(trainX,trainY),
                    error = function(e) {return ("err")})
            if(!is.data.frame(fit)){
                if(fit == "err"){
                    next
                }
            }
            if(nrow(fit) == 0) next
            fit$cg <- cg[i]
            models.ea <- rbind(models.ea,fit)
    }
}

1 


Warning message:
“executing %dopar% sequentially: no parallel backend registered”


2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 
46 
47 
48 
49 


In [47]:
fit

[1] "err"

In [48]:
models.ea <- models.ea[models.ea[,1] != "(Intercept)",]
models.all <- models.all[models.all[,1] != "(Intercept)",]

In [51]:
models.all

NULL

In [49]:
if(!dir.exists(outd)) dir.create(outd)

In [50]:
# mwas by models of all samples
cg2 <- unique(models.all$cg)
mwas.all <- matrix(0,nrow=length(cg2),ncol=2)
for(i in 1:length(cg2)){
        pos <- models.all[models.all$cg == cg2[i],1]
        gwas <- snp.gwas2$z[is.element(snp.gwas2$pos_hg38, pos)]
        weight <- models.all[models.all$cg == cg2[i],2]
        geno <- snp.1kg.eur2[match(pos,map.1kg.eur2$POS),]
        mwas.all[i,] <- MWAS(gwas, weight, t(geno))
}
rownames(mwas.all) <- cg2
colnames(mwas.all) <- c("z","p")

# mwas by models of EA samples
cg2 <- unique(models.ea$cg)
mwas.ea <- matrix(0,nrow=length(cg2),ncol=2)
for(i in 1:length(cg2)){
        pos <- models.ea[models.ea$cg == cg2[i],1]
        gwas <- snp.gwas2$z[is.element(snp.gwas2$pos_hg38, pos)]
        weight <- models.ea[models.ea$cg == cg2[i],2]
        geno <- snp.1kg.eur2[match(pos,map.1kg.eur2$POS),]
        mwas.ea[i,] <- MWAS(gwas, weight, t(geno))
}
rownames(mwas.ea) <- cg2
colnames(mwas.ea) <- c("z","p")

# output models and mwas results
outf <- paste0(outd,"/models-cov1.old.all.wind.",wind[k])
write.csv(models.all,outf)
outf <- paste0(outd,"/models-cov1.old.ea.wind.",wind[k])
write.csv(models.ea,outf)
outf <- paste0(outd,"/mwas-cov1.old.all.wind.",wind[k])
write.csv(mwas.all,outf)
outf <- paste0(outd,"/mwas-cov1.old.ea.wind.",wind[k])
write.csv(mwas.ea,outf)

ERROR: Error in gwas %*% weight: requires numeric/complex matrix/vector arguments


In [ ]:
head(models.ea)

In [ ]:
head(models.ea)

In [ ]:
mwas.all